In [2]:
import numpy as np
import pandas as pd
import scipy

In [3]:
dados = scipy.io.loadmat(r"C:\Users\jmlnn\Downloads\barra_6kv_condi_1.mat")
print(dados.keys())

dict_keys(['__header__', '__version__', '__globals__', 'absoluto'])


In [4]:
dados.values()

dict_values([b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Fri Oct 25 09:54:30 2024', '1.0', [], array([[ 1.26266406e-01,  1.07125781e-01,  9.69695313e-02, ...,
         1.34078906e-01,  1.22164844e-01,  1.38571094e-01],
       [ 2.42780885e+01,  4.71812595e+01,  7.35990319e+01, ...,
         2.06046990e+02,  2.27669605e+02,  2.49815282e+02],
       [ 1.77637807e-02,  2.64073337e-02,  2.51448184e-02, ...,
         1.96421184e-02,  7.16884847e-03,  1.33674614e-02],
       ...,
       [ 1.77637807e-02,  1.18216758e-02,  2.71589820e-02, ...,
         1.23586275e-02,  8.76761038e-03,  2.09166100e-03],
       [ 1.93106094e-02, -9.40774910e-04,  7.01734600e-03, ...,
         3.61843832e-03, -8.24961128e-04,  6.32008617e-03],
       [ 1.77637807e-02,  4.52884681e-03,  1.70881640e-02, ...,
        -3.66505262e-03,  1.35638961e-02,  1.61864116e-02]])])

In [5]:
df = pd.DataFrame(list(dados.values())[-1])
#df

In [6]:
# Criar a coluna de classe
df.loc['classe'] = ['positivo' if valor <= 180 else 'negativo' for valor in df.iloc[1, :]]

In [7]:
# Filtrar os elementos do semi-ciclo positivo
dados_sp = df.loc[:, df.loc['classe'] == 'positivo'].drop('classe').to_numpy()[2:, :]
dados_sp = dados_sp.astype(float)  # Garantir que todos os elementos sejam floats

In [8]:
# Filtrar elementos do semi-ciclo negativo
dados_sn = df.loc[:, df.loc['classe'] == 'negativo'].drop('classe').to_numpy()[2:, :]
dados_sn = dados_sn.astype(float)  # Garantir que todos os elementos sejam floats

In [9]:
matrix = np.array(list(dados.values())[-1])
pulses = matrix[2:, :]

In [10]:
class SignalMetrics():

    def __init__(self, pulses: np.ndarray):
        """
        Inicializa a classe com um array de pulsos.

        Args:
            pulses(ndarray): Array NumPy contendo valores dos pulsos.
        """

        if not isinstance(pulses, np.ndarray): # Validações de entrada
            raise TypeError("Pulses devem ser um array NumPy.")
        if pulses.size == 0:
            raise ValueError("Pulses não pode ser vazio.")
        
        self.pulses = pulses
        self._peak_value = None # prefixo _ indica que são variáveis internas (convenção Python)
        self._rms = None
        self._avg_amplitude = None

    @property # permite acessar o método como se fosse um atributo (sem parênteses)
    def peak_value(self) -> float:
        """Calcula e retorna o valor de pico do sinal"""
        if self._peak_value is None:
            self._peak_value = np.max(np.abs(self.pulses), axis=1)
        return self._peak_value
    
    @property
    def rms(self) -> float:
        """Calcula e retorna o valor RMS do sinal"""
        if self._rms is None:
            self._rms = np.sqrt(np.mean(self.pulses**2, axis=1))
        return self._rms
    
    @property
    def avg_amplitude(self) -> float:
        """Calcula e retorna a amplitude média de cada linha de pulsos"""
        if self._avg_amplitude is None:
            self._avg_amplitude = np.mean(np.abs(self.pulses), axis=1)
        return self._avg_amplitude
    
    @property
    def sqrt_amplitude(self) -> float:
        """Calcula e retorna a raiz quadrada da amplitude média da matriz"""
        return np.sqrt(self.avg_amplitude)

    def get_crest_factor(self) -> float:
        """Calcula e retorna o fator de crista da matriz"""
        return self.peak_value / self.rms
    
    def get_clearance_factor(self) -> float:
        """Calcula e retorna o fator de liberação da matriz"""
        return self.peak_value / self.sqrt_amplitude
    
    def impulse_factor(self) -> float:
        """Calcula e retorna o fator de impulso da matriz"""
        return self.peak_value / self.avg_amplitude
    
    def shape_factor(self) -> float:
        """Calcula e retorna o fator de forma da matriz"""
        return self.rms / self.avg_amplitude
    
    def skewness(self) -> float:
        """Calcula e retorna a assimetria da matriz"""
        return scipy.stats.skew(self.pulses, axis=1)    

    def kurtosis(self) -> float:
        """Calcula e retorna a curtose da matriz"""
        return scipy.stats.kurtosis(self.pulses, axis=1)

In [11]:
# Separação dos dados de treino e teste
from sklearn.model_selection import train_test_split
dados_sp_train, dados_sp_test = train_test_split(dados_sp, test_size=0.2, random_state=42)
dados_sn_train, dados_sn_test = train_test_split(dados_sn, test_size=0.2, random_state=42)

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dados_sp_train_scaled = scaler.fit_transform(dados_sp_train)
dados_sp_test_scaled = scaler.transform(dados_sp_test)
dados_sn_train_scaled = scaler.fit_transform(dados_sn_train)
dados_sn_test_scaled = scaler.transform(dados_sn_test)

In [13]:
metrics_sp = SignalMetrics(dados_sp_train_scaled)

features_sp_train = np.array([
    metrics_sp.avg_amplitude,
    metrics_sp.rms,
    metrics_sp.peak_value,
    metrics_sp.sqrt_amplitude,
    metrics_sp.get_crest_factor(),
    metrics_sp.get_clearance_factor(),
    metrics_sp.impulse_factor(),
    metrics_sp.shape_factor(),
    metrics_sp.skewness(),
    metrics_sp.kurtosis()
]).T

"""
essa transposição da matriz é necessária para que as features sejam as colunas
e as linhas sejam os exemplos
"""

'\nessa transposição da matriz é necessária para que as features sejam as colunas\ne as linhas sejam os exemplos\n'

In [14]:
metrics_sp_test = SignalMetrics(dados_sp_test_scaled)

features_sp_test = np.array([
    metrics_sp_test.avg_amplitude,
    metrics_sp_test.rms,
    metrics_sp_test.peak_value,
    metrics_sp_test.sqrt_amplitude,
    metrics_sp_test.get_crest_factor(),
    metrics_sp_test.get_clearance_factor(),
    metrics_sp_test.impulse_factor(),
    metrics_sp_test.shape_factor(),
    metrics_sp_test.skewness(),
    metrics_sp_test.kurtosis()
]).T

In [15]:
metrics_sn = SignalMetrics(dados_sn_train_scaled)

features_sn_train = np.array([
    metrics_sn.avg_amplitude,
    metrics_sn.rms,
    metrics_sn.peak_value,
    metrics_sn.sqrt_amplitude,
    metrics_sn.get_crest_factor(),
    metrics_sn.get_clearance_factor(),
    metrics_sn.impulse_factor(),
    metrics_sn.shape_factor(),
    metrics_sn.skewness(),
    metrics_sn.kurtosis()
]).T

In [16]:
metrics_sn_test = SignalMetrics(dados_sn_test_scaled)

features_sn_test = np.array([
    metrics_sn_test.avg_amplitude,
    metrics_sn_test.rms,
    metrics_sn_test.peak_value,
    metrics_sn_test.sqrt_amplitude,
    metrics_sn_test.get_crest_factor(),
    metrics_sn_test.get_clearance_factor(),
    metrics_sn_test.impulse_factor(),
    metrics_sn_test.shape_factor(),
    metrics_sn_test.skewness(),
    metrics_sn_test.kurtosis()
]).T

In [17]:
print(f"features_sp_train.shape: {features_sp_train.shape}")
print(f"features_sp_test.shape: {features_sp_test.shape}")
print(f"features_sn_train.shape: {features_sn_train.shape}")
print(f"features_sn_train.shape: {features_sn_test.shape}")


features_sp_train.shape: (320, 10)
features_sp_test.shape: (81, 10)
features_sn_train.shape: (320, 10)
features_sn_train.shape: (81, 10)


In [22]:
# Combine training/test features from both positive and negative semi-cycles 
features_train = np.concatenate((features_sp_train, features_sn_train), axis=0) 
features_test = np.concatenate((features_sp_test, features_sn_test), axis=0)

print(f"Dimensão das features de treino: {features_train.shape}")
print(f"Dimensão das features de teste: {features_test.shape}")

# Create target labels
labels_train = np.array(['positivo'] * features_sp_train.shape[0] + ['negativo'] * features_sn_train.shape[0])
labels_test = np.array(['positivo'] * features_sp_test.shape[0] + ['negativo'] * features_sn_test.shape[0])

print(f"Dimensão dos rótulos de treino: {labels_train.shape}")
print(f"Dimensão dos rótulos de teste: {labels_test.shape}")


Dimensão das features de treino: (640, 10)
Dimensão das features de teste: (162, 10)
Dimensão dos rótulos de treino: (640,)
Dimensão dos rótulos de teste: (162,)


In [25]:
from sklearn.feature_selection import RFE, mutual_info_classif, SelectKBest
from sklearn.linear_model import LogisticRegression

# Initialize the model and RFE
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
rfe = RFE(model, n_features_to_select=5)
features_train_rfe = rfe.fit_transform(features_train, labels_train)
features_test_rfe = rfe.transform(features_test)

# Mutual Information block
selector = SelectKBest(mutual_info_classif, k=5)
features_train_mi = selector.fit_transform(features_train, labels_train)
features_test_mi = selector.transform(features_test)

# COmbine the features selected by RFE and Mutual Information
selected_features_rfe = rfe.get_support(indices=True)
selected_features_mi = selector.get_support(indices=True)
combined_features = np.union1d(selected_features_rfe, selected_features_mi)

# Filter the combined features
features_train_combined = features_train[:, combined_features]
features_test_combined = features_test[:, combined_features]

# Correlation matrix between selected features
correlation_matrix = np.corrcoef(features_train_combined.T)

# Select least correlated features
upper_triangle = np.triu(correlation_matrix, k=1)
to_drop = [i for i in range(upper_triangle.shape[1]) if any(upper_triangle[:, i] > 0.9)]
selected_features_final = [feature for feature in combined_features if feature not in to_drop]

# Filter the final features
features_train_final = features_train[:, selected_features_final]
features_test_final = features_test[:, selected_features_final]

In [26]:
print(f"Shape of features_train_final.T: {features_train_final.T.shape}")
print(f"Shape of labels_train: {labels_train.shape}")

Shape of features_train_final.T: (4, 640)
Shape of labels_train: (640,)


In [30]:
from sklearn.metrics import accuracy_score, classification_report

model.fit(features_train_final, labels_train)
predictions = model.predict(features_test_final)

accuracy = accuracy_score(['positivo'] * features_sp_test.shape[0] + ['negativo'] * features_sn_test.shape[0], predictions)
report = classification_report(['positivo'] * features_sp_test.shape[0] + ['negativo'] * features_sn_test.shape[0], predictions)


In [33]:
print(report)

              precision    recall  f1-score   support

    negativo       0.52      0.84      0.64        81
    positivo       0.57      0.21      0.31        81

    accuracy                           0.52       162
   macro avg       0.54      0.52      0.47       162
weighted avg       0.54      0.52      0.47       162

